In [1]:
import os

import playsound as playsound
from keybert import KeyBERT
import whisper
from pydub import AudioSegment
import urllib.request
import math
import openai
import consts
from stable_whisper import results_to_sentence_word_ass
from pydub import AudioSegment
from pydub.silence import split_on_silence
from queue import Queue

playsound is relying on a python 2 subprocess. Please use `pip3 install PyObjC` if you want playsound to run more efficiently.


In [6]:
# Configurations

# anything under SILENT_TRESH dBFS is considered silence
# if clips duration are so long, SILENT_TRESH must be a lower value (between 60 an 85 must be ok)
SILENT_TRESH = 60
MAX_LOCAL_TOPICS = 4


# AUDIO_URL = "https://stakwork-uploads-dev.s3.amazonaws.com/short_ep_48_kevin_rooke.mp3"
AUDIO_URL = "https://soundcloud.com/bitcoin-podcast/corb-lund-the-bitcoin-cowboy?si=aea5659804774d60827f98bd5f4db427&utm_source=clipboard&utm_medium=text&utm_campaign=social_sharing"

# Configuration
LANGUAGE = "English"

filebasename = "audio"

#inits
model = whisper.load_model("base")
kw_model = KeyBERT()

# Download audio file
urllib.request.urlretrieve(AUDIO_URL, filebasename + ".mp3")

# Convert audio file
# sound = AudioSegment.from_mp3(filebasename + ".mp3")
# sound = AudioSegment.from_mp3("bitcoin" + ".mp3")
sound = AudioSegment.from_mp3("btccowboy" + ".mp3")
sound.export(filebasename + ".wav", format="wav")

# transcript = model.transcribe("audio.wav",fp16=False)
# transcriptGlobal = model.transcribe('audio.wav', fp16=False, max_initial_timestamp=None, verbose=True)
#transcriptGlobal = model.transcribe('audio.wav', fp16=False, max_initial_timestamp=None, verbose=False, language=LANGUAGE)



<_io.BufferedRandom name='audio.wav'>

In [8]:



WORDS_PER_TOPIC = 1


# Configuration
MAX_LOCAL_TOPICS = 4



#remove old chunks in folder
mydir="./"
for f in os.listdir(mydir):
    if not f.startswith("chunk"):
        continue
    os.remove(f)

#Spittin audio in silences
finalList = []
print("THIS PROCESS COULD TAKE LONG, PLEASE BE PATIENT!")
audio_chunks = split_on_silence(sound, silence_thresh=-SILENT_TRESH)
# audio_chunks = split_on_silence(sound, min_silence_len=MIN_SILENT_LENGTH, silence_thresh=-SILENT_TRESH)
#audio_chunks = split_on_silence(sound, min_silence_len=MIN_SILENT_LENGTH, silence_thresh=-SILENT_TRESH, keep_silence=KEEP_SILENCE)
print("DONE!")



THIS PROCESS COULD TAKE LONG, PLEASE BE PATIENT!
DONE!


In [9]:
totalduration = 0
for chunk in audio_chunks:
    totalduration += chunk.duration_seconds
print("original sound duration:", sound.duration_seconds)
print("totalduraton:", totalduration)

original sound duration: 2065.031836734694
totalduraton: 2012.0889795918372


In [ ]:
chunk_topic_list=[]
for i, chunk in enumerate(audio_chunks):
   out_file = "chunk{0}.wav".format(i)
   chunk.export(out_file, format="wav")
   transcriptLocal = model.transcribe(out_file, fp16=False, max_initial_timestamp=None, verbose=False, language=LANGUAGE)
   keywordsLocal = kw_model.extract_keywords(transcriptLocal["text"], top_n=MAX_LOCAL_TOPICS)
   local_topics=[]
   for j in range(len(keywordsLocal)):
       if j <= MAX_LOCAL_TOPICS:
          local_topics.append(keywordsLocal[j][0])

   print("chunk ", i , " duration ", chunk.duration_seconds, " topics: ",local_topics )
   chunk_topic_list.append([i,chunk,local_topics])


100%|██████████| 8010/8010 [00:07<00:00, 1143.05frames/s]


chunk  0  duration  80.10498866213152  topics:  ['corb', 'bitcoin', 'podcast', 'listening']


100%|██████████| 1769/1769 [00:02<00:00, 776.33frames/s]


chunk  1  duration  17.697006802721088  topics:  ['music', 'london', 'corb', 'doing']


100%|██████████| 9321/9321 [00:10<00:00, 925.52frames/s] 


chunk  2  duration  93.21600907029479  topics:  ['music', 'radio', 'songwriting', 'underground']


  0%|          | 0/6124 [00:00<?, ?frames/s]

In [123]:
combined_segments = []
for i,chunk,topics in chunk_topic_list:
    print("chunk ", i , " duration ", chunk.duration_seconds, " topics: ",topics )


chunk  0  duration  80.10498866213152  topics:  ['corb', 'bitcoin', 'podcast', 'listening']
chunk  1  duration  17.697006802721088  topics:  ['music', 'london', 'corb', 'doing']
chunk  2  duration  93.21600907029479  topics:  ['music', 'radio', 'songwriting', 'underground']
chunk  3  duration  61.2409977324263  topics:  ['audience', 'culturally', 'diverse', 'music']
chunk  4  duration  14.63498866213152  topics:  ['canada', 'japan', 'australia', 'uk']
chunk  5  duration  156.40201814058958  topics:  ['music', 'listening', 'bluegrass', 'rock']
chunk  6  duration  6.458004535147392  topics:  ['democratic', 'fork', 'source', 'really']
chunk  7  duration  5.618004535147392  topics:  ['watch', 'interesting', 'going', 'think']
chunk  8  duration  39.71  topics:  ['bitcoin', 'cryptocurrencies', 'investing', 'future']
chunk  9  duration  86.6780045351474  topics:  ['casino', 'cryptocurrencies', 'transactions', 'vegas']
chunk  10  duration  146.33700680272108  topics:  ['technologies', 'mp3s', 

In [124]:
#remove old segments in folder
mydir="./"
for f in os.listdir(mydir):
    if not f.startswith("segment"):
        continue
    os.remove(f)

#find short chunks and combine with previous if is same topic
combined_segments = []
segment_list=[]
for i,chunk,topics in chunk_topic_list:
    #print("chunk ", i , " duration ", chunk.duration_seconds, " topics: ",topics )
    if chunk.duration_seconds < 60:
        print(i, "shorter than 60secs")
        # if i > 0 and i < len(segments) - 1:
        if 0 < i < len(chunk_topic_list) - 1:
            #if topics are in combined_segments[-1] topic list
            previous_topics = chunk_topic_list[i-1][2]
            print(i, "topics", topics)
            print(i, "previous topics ", previous_topics)
            topic_found = False
            for local_topic in topics:
                if local_topic in previous_topics:
                    topic_found = True

            if topic_found:
                print(i," same topic as previous")
                combined_segments[-1] = combined_segments[-1] + chunk #+ chunk_topic_list[i+1]
                print("combining ", i ," with ", i-1)
                for j, combi_seg in enumerate(combined_segments):
                    print(j, " ",combi_seg.duration_seconds)
            else:
                print(i,"NOT same topic as previous")
        else:
            print(i, "is 0 or ", len(chunk_topic_list) )
            combined_segments.append(chunk)
            print("combined_segments", i , " duration ", chunk.duration_seconds)
            chunk.export("segment_{}{}.wav".format(i,topics), format="wav")
            print("segment_{}.wav".format(i), " duration ", chunk.duration_seconds)
            segment_list.append([i,chunk,topics])
    else:
        combined_segments.append(chunk)
        chunk.export("segment_{}{}.wav".format(i,topics), format="wav")
        print("segment_{}.wav".format(i), " duration ", chunk.duration_seconds)
        segment_list.append([i,chunk,topics])



segment_0.wav  duration  80.10498866213152
1 shorter than 60secs
1 topics ['music', 'london', 'corb', 'doing']
1 previous topics  ['corb', 'bitcoin', 'podcast', 'listening']
1  same topic as previous
combining  1  with  0
0   97.80199546485261
segment_2.wav  duration  93.21600907029479
segment_3.wav  duration  61.2409977324263
4 shorter than 60secs
4 topics ['canada', 'japan', 'australia', 'uk']
4 previous topics  ['audience', 'culturally', 'diverse', 'music']
4 NOT same topic as previous
segment_5.wav  duration  156.40201814058958
6 shorter than 60secs
6 topics ['democratic', 'fork', 'source', 'really']
6 previous topics  ['music', 'listening', 'bluegrass', 'rock']
6 NOT same topic as previous
7 shorter than 60secs
7 topics ['watch', 'interesting', 'going', 'think']
7 previous topics  ['democratic', 'fork', 'source', 'really']
7 NOT same topic as previous
8 shorter than 60secs
8 topics ['bitcoin', 'cryptocurrencies', 'investing', 'future']
8 previous topics  ['watch', 'interesting', '

In [125]:
for i,chunk,topics in segment_list:
    if chunk.duration_seconds < 60:
        print(i, "Should not be any. Maybe last one!")
    else:
        print(i, " duration:", chunk.duration_seconds, " topics", topics)


0  duration: 80.10498866213152  topics ['corb', 'bitcoin', 'podcast', 'listening']
2  duration: 93.21600907029479  topics ['music', 'radio', 'songwriting', 'underground']
3  duration: 61.2409977324263  topics ['audience', 'culturally', 'diverse', 'music']
5  duration: 156.40201814058958  topics ['music', 'listening', 'bluegrass', 'rock']
9  duration: 86.6780045351474  topics ['casino', 'cryptocurrencies', 'transactions', 'vegas']
10  duration: 146.33700680272108  topics ['technologies', 'mp3s', 'ip3s', 'tech']
12  duration: 85.36600907029478  topics ['uber', 'bitcoin', 'cryptocurrencies', 'banking']
13  duration: 104.8  topics ['bitcoin', 'coin', 'odds', 'funny']
14  duration: 65.1809977324263  topics ['debt', 'globalized', 'trillion', 'ledger']
16  duration: 63.3809977324263  topics ['venezuela', 'euro', 'bitcoin', 'greece']
24  duration: 97.42501133786848  topics ['slowing', 'core', 'lag', 'transactions']
26  duration: 119.45897959183674  topics ['bitcoin', 'cryptocurrencies', 'slowe

In [127]:
for i,chunk,topics in segment_list:
    if chunk.duration_seconds < 60:
        print(i, "Should not be any. Maybe last one!")
    elif chunk.duration_seconds >= 300:
        print("----->",i, " duration:", chunk.duration_seconds, " topics", topics)


40 Should not be any. Maybe last one!
